# 제 16장 법 m에 대한 거듭제곱과 연속제곱법
$a^k$ (mod m) 계산

**** PYTHON 내 담을 수 있는 숫자의 길이제한 때문에 너무 큰 수에 대해서는 제대로 작동 X

In [1]:
import numpy as np
import pandas as pd

def power_mod_cal(a,k,m):
    
    # 1) a의 2^k 거듭제곱을 법 m에 대해 계산한 결과표
    a_power2_df = pd.DataFrame(index=range(0))
    for i in range(int(np.log2(k))+1) :
        if i == 0 : 
            value = a**(i+1) % m
        else :
            value = (value**2) % m
        tmp_df = pd.DataFrame([[str(2**(i)), str(a) + '^' + str(2**(i)), value]])
        a_power2_df = pd.concat([a_power2_df,tmp_df])
    a_power2_df.columns = ['power_num','real_value', 'mod_value']
    a_power2_df.reset_index(drop=True, inplace=True)
    
    # 2) k의 2진법 전개
    binary_expansion = bin(k)[2:]
    
    # 3) 2진법 전개 활용한 합동수 계산
    mod_val = np.float64(1)
    for i in range(len(binary_expansion)) :
        if binary_expansion[i] == '1' :
            mod_val *= a_power2_df['mod_value'].iloc[-(i+1)]
            mod_val = mod_val % m
    return int(mod_val)

In [68]:
a = 7
k = 327
m = 853
power_mod_cal(a=a, k=k, m=m)

286

In [ ]:
(7**(10**200000)) % 853

In [48]:
a = 5
k = 117
m = 19
power_mod_cal(a=a, k=k, m=m)

1

In [45]:
a = 7
k = float(10**(200000))
m = 853
power_mod_cal(a=a, k=k, m=m)

OverflowError: int too large to convert to float

# 제 17장 법 m에 대한 k-제곱근 구하기
$x^k$ $\equiv$ b (mod m) 계산

where gcd(b, m ) = 1, gcd(k, $\Phi$(m)) = 1

참고) 오일러의 공식 : $a^{\phi (m)}$ $\equiv$ 1 (mod m)  where gcd(a,m) = 1, s.t $\phi (m)$ = #{a : 1 $\le$ a $\le$ m, gcd(a,m) = 1}  ( 1과 m 사이 수 중 m과 서로소인 수의 개수)

**** PYTHON 내 담을 수 있는 숫자의 길이제한 때문에 너무 큰 수에 대해서는 제대로 작동 X

In [2]:
import numpy as np
from collections import Counter
def root_mod_cal(k,b,m):
    
    # 가정 위배되는 경우 에러 발생(1)
    if np.gcd(b,m) != 1 :
        raise NameError('gcd(b,m) != 1')
        
    # 소인수 계산
    def factorize2(n):
        factor = 2 #시작 소수 지정
        factors = []
        while (factor**2 <= n):  # 루트n까지 실행
            while (n % factor == 0):  # 소수로 나누어 떨어지면(= 즉 약수면)
                factors.append(factor)  # 리스트에 추가
                n = n // factor  # n을 몫으로 변경
            factor += 1
        if n > 1 : # 1보다 크고 factor**2(4)보다 작은 경우 n은 소수임으로 append -> 2,3 경우
            factors.append(n)
        return factors

    # pi(m)계산
    fac_dic = Counter(factorize2(m))
    pi_m = 1
    for i in fac_dic.keys() :
        pi_m *= (i ** fac_dic[i] - i ** (fac_dic[i]-1))
    
    # 가정 위배되는 경우 에러 발생(2)
    if np.gcd(k,pi_m) != 1 :
        raise NameError('gcd(k,pi(m)) != 1')
    
    # ku - pi(m)v = 1의 양수해 계산 (참고 : https://madrabbit7.tistory.com/96)
    def congruence_equation(a, c, m):
        g = np.gcd(a, m)  
        lst = []
        if c % g != 0:
            print(f"gcd({a}, {m}) = {g} : {c}을 나눌 수 없습니다. 정수해가 없습니다.")
            return None
        da = a // g  # 3
        dc = c // g  # 2
        dm = m // g  # 13
        for u in range(0, dm):  # 대입해가며 계산
            r = (da * u - dc) % dm

            if r ==0:

                break
        v = int((k * u - 1) / m)
        return u, v

    u,v =congruence_equation(k,1,pi_m) # 항상 양수해

    # 합동식 처리
    # (x^k)^u -> x^(  beta + pi(m)*alpha  )
    # 오일러의 공식 사용하기 위해
    alpha = k * u // pi_m
    beta = k * u % pi_m

    # 연속제곱법 식으로 변경
    #a^k(mod m)
    a = b
    k = u
    m = m

    # 연속제곱법 사용해 x와 합동인 값 산출
    result = power_mod_cal(a=a,k=k,m=m)
    return result

In [20]:
# 제대로 작동
k = 131
b = 758
m = 1073
root_mod_cal(k=k, b=b, m=m)

905

In [25]:
# <ipython-input-24-65bf6d823c6f>:43: RuntimeWarning: overflow encountered in long_scalars 발생
# 길이가 긴 값을 담을 수 없어 제대로 된 답이 나오지 않음 ( 원래 답 : 22,929,826)
k = 3968039
b = 74781
m = 27040397
root_mod_cal(k=k, b=b, m=m)

<ipython-input-24-65bf6d823c6f>:43: RuntimeWarning: overflow encountered in long_scalars
  r = (da * u - dc) % dm


24624353

# 제 19장 라빈-밀러 합성수 판정법
n이 홀수, 적당한 홀수 q에 대해 n-1 = $2^k$q로 표현하자. n의 배수가 아닌 적당한 a가 다음 두 조건을 만족한다면 n은 합성수이다.

(a) $a^q$ !$\equiv$ 1 (mod n)

(b) 모든 i = 0, 1, ..., k-1에 대해 $a^{(2^i)}q$ !$\equiv$ -1 (mod n)

In [3]:
def rabin_miller_test_for_composite_numbers(n) :
    
    # k 계산
    k = 0
    for _k in range(1, int(np.log2(n))+1):
        if (n-1)%(2**(_k)) == 0 :
            k = _k
            
    # q 계산
    q = int((n-1) / (2**k))
    
    # 조건만족여부 확인
    # 만일 n이 홀수인 합성수라면, 1과 n-1 사이에 적어도 75%의 수가 라빈-밀러 증인
    def rabin_miller_witness(a,q,n) :
        if power_mod_cal(a=a, k=q, m=n) != 1 : # (a) 조건
            for i in range(k):   # (b) 조건
                if i == 0 :
                    tmp = power_mod_cal(a, q, n)
                else :
                    tmp = tmp**2 % n

                if tmp % n == (n-1) :
                    return None
        else :
            return None
        
        return a
    
    # 라빈-밀러 증인 리스트 생성
    rabin_miller_witness_lst = []
    
    for a in range(1, n):
        tmp = rabin_miller_witness(a,q,n)
        if tmp != None :
            print(f"라빈-밀러 증인 {tmp}를 찾았습니다")
            print(f"{n}은 합성수입니다.")
            return None
    print(f"{n}은 굉장히 높은 확률(1 - 0.25^100)로 소수입니다.")
    return None

In [4]:
rabin_miller_witness_lst = rabin_miller_test_for_composite_numbers(561)

라빈-밀러 증인 2를 찾았습니다
561은 합성수입니다.


In [5]:
rabin_miller_test_for_composite_numbers(172947529)

라빈-밀러 증인 2를 찾았습니다
172947529은 합성수입니다.


In [6]:
rabin_miller_test_for_composite_numbers(7)

7은 굉장히 높은 확률(1 - 0.25^100)로 소수입니다.


In [ ]:
# n보다 작은 라빈-밀러 증인을 모두 찾아 저장하려 했으나 코드 수행 시간이 너무 길어 증인 1개를 찾으면 코드 종료 및 문구 표시하도록 수정